# Solidity语法

熟悉solidity languag，并持续更补基础语法

声明语法版本，后续省略
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;
```

## 数据与函数
数据类型与函数

### uint

- uint8   ranges from 0 to 2 ** 8 - 1
- uint16  ranges from 0 to 2 ** 16 - 1
- ...
- uint256 ranges from 0 to 2 ** 256 - 1, default

### int
- int256 ranges from -2 ** 255 to 2 ** 255 - 1, default
- int128 ranges from -2 ** 127 to 2 ** 127 - 1
- int public minInt = type(int).min;
- int public maxInt = type(int).max;

### address
- public addr = 0xCA35b7d915458EF540aDe6068dFe2F44E8fa733c;


### byte(byte[])

- bytes1 a = 0xb5; //  [10110101]

### Default values
- bool public defaultBoo; // false
- uint public defaultUint; // 0
- int public defaultInt; // 0
- address public defaultAddr; // 0x0000000000000000000000000000000000000000



### Mapping

- mapping(address => uint) public myMap;
  1. keyType：string、address等
  2. valueType：anyType
  3. 空则返回默认值

#### example
```solidity
contract Mapping {
    // Mapping from address to uint
    mapping(address => uint) public myMap;

    function get(address _addr) public view returns (uint) {
        return myMap[_addr];
    }

    function set(address _addr, uint _i) public {
        myMap[_addr] = _i;
    }
    function remove(address _addr) public {
        delete myMap[_addr];
    }
}
```

### Array

#### example
```solidity

// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;

contract Array {
    // Several ways to initialize an array
    uint[] public arr;
    uint[] public arr2 = [1, 2, 3];
    uint[10] public myFixedSizeArr; //固定size，default value = 0

    function get(uint i) public view returns (uint) {
        return arr[i];
    }

    // not recommend
    function getArr() public view returns (uint[] memory) {
        return arr;
    }
    // append ele
    function push(uint i) public {
        arr.push(i);
    }
    // pop ele
    function pop() public {
        arr.pop();
    }
    function getLength() public view returns (uint) {
        return arr.length;
    }
    // delete means set 0
    function remove(uint index) public {
        delete arr[index];
    }
    function examples() external {
        // 内存创建，仅能固定大小
        uint[] memory a = new uint[](5);
    }
}
```



### Enum
方便模式选择和状态跟随，支持contract导入。类似`proto`

#### example
```solidity
contract Enum {
    // 状态转移
    enum Status {
        Pending, //0
        Shipped, //1
        Accepted,//2
        Rejected,//3
        Canceled //4
    }
    // 默认声明第一个status
    Status public status;
    function get() public view returns (Status) {
        return status;
    }

    // 传入uint更新状态
    function set(Status _status) public {
        status = _status;
    }

    // func对应status转换
    function cancel() public {
        status = Status.Canceled;
    }
    // 状态置0
    function reset() public {
        delete status;
    }
}

```

### Structs
创建结构体自定义类型，整体组织相关数据，类似Enum定义支持contract导入

#### example
```solidity
contract Todos {
    struct Todo {
        string text;
        bool completed; //default false
    }
    Todo[] public todos; // 声明struct array

    function create(string calldata _text) public {
        todos.push(Todo(_text, false)); //1. func easy create, calling it like a function
        todos.push(Todo({text: _text, completed: false})); //2. map create, key value mapping
        Todo memory todo;  // 3. initialize an empty struct and then update it
        todo.text = _text;
        todos.push(todo);
    }

    function get(uint _index) public view returns (string memory text, bool completed) {
        Todo storage todo = todos[_index];
        return (todo.text, todo.completed);
    }

    // update text for todos[_index]
    function updateText(uint _index, string calldata _text) public {
        Todo storage todo = todos[_index];
        todo.text = _text;
    }

    // update completed
    function toggleCompleted(uint _index) public {
        Todo storage todo = todos[_index];
        todo.completed = !todo.completed;
    }
}
```

### Function


#### example
```solidity
contract Function {
    // Functions can return multiple values.
    function returnMany() public pure returns (uint x, bool b, uint y){ //可不需指定return name
        return (1, true, 2);

        //可对var name赋值作为return
        //x = 1;
        //b = true;
        //y = 2;
    }
    (uint i, bool b, uint j) = returnMany(); //指定dataType

    // 不可传入传出map

    // 可传入array
    function arrayInput(uint[] memory _arr) public {}

    // return state variables
    uint[] public arr;
    function arrayOutput() public view returns (uint[] memory) {
        return arr;
    }
}
```

### View and Pure Functions

- `View function` 不改变状态，
- `Pure function` 不改变状态，也不读

#### example
```solidity
contract ViewAndPure {
    uint public x = 1;
    function addToX(uint y) public view returns (uint) { // 不改变，状态
        return x + y;
    }
    function add(uint i, uint j) public pure returns (uint) { // 不改变，不读 状态
        return i + j;
    }
}
```

### function modifier
某个函数调用前后执行的code，功能：
- Restrict access（准入限制）
- Validate inputs（验证输入）
- Guard against reentrancy hack（重入hack防御）

#### example
```solidity
contract FunctionModifier {
    address public owner;
    uint public x = 10;
    bool public locked;
    constructor() {
        owner = msg.sender;
    }
    // Modifier：检查caller是否是onwer
    modifier onlyOwner() {
        require(msg.sender == owner, "Not owner");
        _; //再执行function
    }
    // Modifiers：检查输入非0
    modifier validAddress(address _addr) {
        require(_addr != address(0), "Not valid address");
        _;
    }
    function changeOwner(address _newOwner) public onlyOwner validAddress(_newOwner) {
        owner = _newOwner;
    }

    // Modifiers：函数锁
    modifier noReentrancy() {
        require(!locked, "No reentrancy");
        locked = true;
        _;
        locked = false;
    }

    function decrement(uint i) public noReentrancy {
        x -= i;
        if (i > 1) {
            decrement(i - 1);
        }
    }
}


```

### Constructor
执行于contract之上，可选的function

#### example
```solidity
// Base contract X
contract X {
    string public name;
    constructor(string memory _name) {
        name = _name;
    }
}
// Base contract Y
contract Y {
    string public text;
    constructor(string memory _text) {
        text = _text;
    }
}
// contract继承初始化
// 1. Pass the parameters here in the inheritance list.
contract B is X("Input to X"), Y("Input to Y") {}
// 2. similar to function modifiers.
contract C is X, Y {
    constructor(string memory _name, string memory _text) X(_name) Y(_text) {}
}

// 按照继承顺序
contract D is X, Y {
    constructor() X("X was called") Y("Y was called") {}
}
contract E is X, Y {
    constructor() Y("Y was called") X("X was called") {}
}



```

### Variables

- local（本地变量）
  1. 函数内声明
  2. 不存储于链中

- state（状态变量）
  1. 函数外声明
  2. 存储于链中

- global（全局变量）
  1. 链中上下文信息

#### example
```solidity
// SPDX-License-Identifier: MIT
pragma solidity ^0.8.13;

contract Variables {
    // State variables are stored on the blockchain.
    string public text = "Hello";
    uint public num = 123;

    function doSomething() public {
        // Local variables are not saved to the blockchain.
        uint i = 456;

        // Here are some global variables
        uint timestamp = block.timestamp; // Current block timestamp
        address sender = msg.sender; // address of the caller
    }
}

```

#### constant（常量）
大写声明常量类型，节省gas cost

- address public constant MY_ADDRESS = 0x777788889999AaAAbBbbCcccddDdeeeEfFFfCcCc;
- uint public constant MY_UINT = 123;

#### Immutable（不可变量）
和constant相似，但是允许构造函数中修改。示例：

```solidity
contract Immutable {
    address public immutable MY_ADDRESS;
    uint public immutable MY_UINT;
    constructor(uint _myUint) {
        MY_ADDRESS = msg.sender;
        MY_UINT = _myUint;
    }
}
```


#### 变量-读与写
- 变量的写和更新，需要发送交易
- 变量的读，不需要发送交易和交易费用

### Data Locations
Variables声明storage、memory、calldata指定数据地址。

- storage：state variable (状态变量存储于blockchain)
- memory：内存中函数生命周期调用
- calldata：指定函数参数存储

#### 示例
```solidity
contract DataLocations {
    uint[] public arr;
    mapping(uint => address) map;
    struct MyStruct {
        uint foo;
    }
    mapping(uint => MyStruct) myStructs;

    function f() public {
        // 函数调用state variables。在contract内声明，默认storage state variable
        _f(arr, map, myStructs[1])；
        MyStruct storage myStruct = myStructs[1];
        MyStruct memory myMemStruct = MyStruct(0); // create a struct in memory
    }
    function _f(
        uint[] storage _arr,
        mapping(uint => address) storage _map,
        MyStruct storage _myStruct
    ) internal {
        // do something with storage variables
    }
    function g(uint[] memory _arr) public returns (uint[] memory) { // return memory variables
        // do something with memory array
    }
    function h(uint[] calldata _arr) external {
        // do something with calldata array
    }
}
```

### Visibility
函数和状态变量必须声明是否可被其他contract接入

- Functions
  1. public， 任何contract和account都可以（所有）
  2. external，仅外部contract和account可以（外部）
  3. private， contract内部定义的function（contract私有）
  4. internal，继承`internal`function的contract（同支contract可用）
 

- State variables
  1. public
  2. private
  3. internal

#### example
```solidity
contract Base {
    // Private function在contract内部使用，子contract不可使用
    function privateFunc() private pure returns (string memory) {
        return "private function called";
    }
    function testPrivateFunc() public pure returns (string memory) {
        return privateFunc();
    }

    // Internal function can be called
    // - contract内部
    // - 子contracts内部
    function internalFunc() internal pure returns (string memory) {
        return "internal function called";
    }
    function testInternalFunc() public pure virtual returns (string memory) {
        return internalFunc();
    }

    // Public functions can be called
    // - contract内部
    // - 子contract内部
    // - 其他contracts and accounts
    function publicFunc() public pure returns (string memory) {
        return "public function called";
    }

    // External functions can only be called
    // - 仅其他contracts and accounts
    function externalFunc() external pure returns (string memory) {
        return "external function called";
    }

    // This function will not compile since we're trying to call
    // an external function here.
    // function testExternalFunc() public pure returns (string memory) {
    //     return externalFunc();
    // }

    // State variables
    string private privateVar = "my private variable";
    string internal internalVar = "my internal variable";
    string public publicVar = "my public variable";
    // State variables cannot be external so this code won't compile.
    // string external externalVar = "my external variable";
}

contract Child is Base {
    // 子contracts 不能使用private functions，state variables.
    // function testPrivateFunc() public pure returns (string memory) {
    //     return privateFunc();
    // }

    // Internal function call be called inside child contracts.
    function testInternalFunc() public pure override returns (string memory) {
        return internalFunc();
    }
}



```

## 关键字

### Payable
Functions and addresses声明为`payable` 接收ether

```solidity
contract Payable {
    address payable public owner; // 可支付address
    constructor() payable { // Payable constructor can receive Ether
        owner = payable(msg.sender); 
    }

    // Function支持payable
    function deposit() public payable {}
    // 不支持 payable
    function notPayable() public {}
    // 从contract赎回ether
    function withdraw() public {
        uint amount = address(this).balance;
        (bool success, ) = owner.call{value: amount}("");
        require(success, "Failed to send Ether");
    }
    // 交易ether，_to声明为payable
    function transfer(address payable _to, uint _amount) public {
        (bool success, ) = _to.call{value: _amount}("");
        require(success, "Failed to send Ether");
    }
}
```



### Fallback
无输入参数，无返回值。执行时：
- 不存在函数被调用时
- receive() 不存在或者`msg.data`非空时，发送ether时

被`transfer`或者`send`调用时`fallback`有2300 gas的限制。

### Call
与其它contracts交互的函数，调用fallback function，用`call`发送ether，调用已存在函数时不建议。

### Delegatecall
类似`call`。contract A执行delegatecall对contract B,执行B's code，用contract A's storage, msg.sender和msg.value。

**TODO**

### Function Selector
calldata的前4 bytes指定了函数，这被称为`function selector`。

用`call`执行`transfer `
```solidity
addr.call(abi.encodeWithSignature("transfer(address,uint256)", 0xSomeAddress, 123))
```
返回的前4 bytes就是`function selector`。

### Events
- 监听events更新用户界面
- 节省存储

#### example
```solidity
contract Event {
    // Up to 3 parameters can be indexed, filter log by address
    event Log(address indexed sender, string message);
    event AnotherLog();
    function test() public {
        emit Log(msg.sender, "Hello World!");
        emit Log(msg.sender, "Hello EVM!");
        emit AnotherLog();
    }
}
```

### Try Catch
try / catch仅能捕获external function calls和contract creation的发生错误

### Import
导入本地和外部files

#### Local
```
├── Import.sol
└── Foo.sol
```
FOO.sol

```solidity
struct Point {
    uint x;
    uint y;
}
error Unauthorized(address caller);
function add(uint x, uint y) pure returns (uint) {
    return x + y;
}
contract Foo {
    string public name = "Foo";
}
```

Import.sol
```solidity
// import Foo.sol from current directory
import "./Foo.sol";

// import {symbol1 as alias, symbol2} from "filename";
import {Unauthorized, add as func, Point} from "./Foo.sol";
contract Import {
    // Initialize Foo.sol
    Foo public foo = new Foo();

    // Test Foo.sol by getting it's name.
    function getFooName() public view returns (string memory) {
        return foo.name();
    }
}
```


#### External
```solidity
// https://github.com/owner/repo/blob/branch/path/to/Contract.sol
import "https://github.com/owner/repo/blob/branch/path/to/Contract.sol";

// Example import ECDSA.sol from openzeppelin-contract repo, release-v4.5 branch
// https://github.com/OpenZeppelin/openzeppelin-contracts/blob/release-v4.5/contracts/utils/cryptography/ECDSA.sol
import "https://github.com/OpenZeppelin/openzeppelin-contracts/blob/release-v4.5/contracts/utils/cryptography/ECDSA.sol";

```

### ABI Decode
- `abi.encode` 将data编码为bytes
- `abi.decode` 将bytes解码为data

```solidity
abi.encode(x, addr, arr, myStruct);

(x, addr, arr, myStruct) = abi.decode(data, (uint, address, uint[], MyStruct));
```

### Verifying Signature

Messages签署chain并且用contract在链上验证。

### Error
交易中error，撤销已改状态。`error`类型包括：**require, revert or assert**

- `require` 执行前验证
- `revert` 类似require
- `assert` 代码检查。控制流程节省gas

#### example
```solidity
contract Error {
    function testRequire(uint _i) public pure {
        require(_i > 10, "Input must be greater than 10"); // Require  用于检查 inputs、执行前条件、其他函数返回等
    }
    function testRevert(uint _i) public pure {
        if (_i <= 10) {
            revert("Input must be greater than 10"); //复杂使用
        }
    }
    uint public num;
    function testAssert() public view {
        assert(num == 0); // Assert 用于检查internal errors。view声明function不改变状态，故检查num值的改动
    }
    // 自定义声明error
    error InsufficientBalance(uint balance, uint withdrawAmount);
    function testCustomError(uint _withdrawAmount) public view {
        uint bal = address(this).balance;
        if (bal < _withdrawAmount) {
            revert InsufficientBalance({balance: bal, withdrawAmount: _withdrawAmount});
        }
    }
}
```

### Calling Other Contract
contract互相调用有两种方式：

- A.foo(x, y, z).
- call. 不被建议

```solidity
contract Callee {
    uint public x;
    uint public value;
    function setX(uint _x) public returns (uint) {
        x = _x;
        return x;
    }
    function setXandSendEther(uint _x) public payable returns (uint, uint) {
        x = _x;
        value = msg.value;
        return (x, value);
    }
}
contract Caller {
    function setX(Callee _callee, uint _x) public {
        uint x = _callee.setX(_x);
    }
    function setXFromAddress(address _addr, uint _x) public {
        Callee callee = Callee(_addr);
        callee.setX(_x);
    }
    function setXandSendEther(Callee _callee, uint _x) public payable {
        (uint x, uint value) = _callee.setXandSendEther{value: msg.value}(_x);
    }
}
```

### Contract that Creates other Contracts
contract创建contract

```solidity
contract Car {
    address public owner;
    string public model;
    address public carAddr;
    constructor(address _owner, string memory _model) payable {
        owner = _owner;
        model = _model;
        carAddr = address(this);
    }
}
contract CarFactory {
    Car[] public cars;
    function create(address _owner, string memory _model) public {
        Car car = new Car(_owner, _model);
        cars.push(car);
    }
    function createAndSendEther(address _owner, string memory _model) public payable {
        Car car = (new Car){value: msg.value}(_owner, _model);
        cars.push(car);
    }
    function create2(address _owner,string memory _model,bytes32 _salt) public {
        Car car = (new Car){salt: _salt}(_owner, _model);
        cars.push(car);
    }
    function create2AndSendEther(address _owner,string memory _model,bytes32 _salt) public payable {
        Car car = (new Car){value: msg.value, salt: _salt}(_owner, _model);
        cars.push(car);
    }
    function getCar(uint _index)public view returns (address owner,string memory model,address carAddr,uint balance){
        Car car = cars[_index];
        return (car.owner(), car.model(), car.carAddr(), address(car).balance);
    }
}
```

### Contract Inheritance
Solidity用`is`支持多重继承。
- 被子contract重写的function
  1. 父contract声明`virtual`
  2. 子contract声明`virtual override`

#### example

```solidity
/* Graph of inheritance
    A
   / \
  B   C
 / \ /
F  D,E
*/

contract A {
    function foo() public pure virtual returns (string memory) {
        return "A";
    }
}
// B继承A
contract B is A { // Override A.foo()
    function foo() public pure virtual override returns (string memory) {
        return "B";
    }
}
// C继承A
contract C is A { // Override A.foo()
    function foo() public pure virtual override returns (string memory) {
        return "C";
    }
}

// 同一函数被多contract重复定义, 从右到左确定right to left,

contract D is B, C {
    // D.foo() returns "C"，C最右
    function foo() public pure override(B, C) returns (string memory) {
        return super.foo();
    }
}
contract E is C, B {
    // E.foo() returns "B"，B最右
    function foo() public pure override(C, B) returns (string memory) {
        return super.foo();
    }
}

// A、B顺序保持一致
contract F is A, B {
    function foo() public pure override(A, B) returns (string memory) {
        return super.foo();
    }
}
```

#### Shadowing Inherited State Variables
state variables 继承后不能被重写

#### Calling Parent Contracts
用`super`调用父contract的方法

## 其他类型

### Library
类似库函数，一般被嵌入到contract中，或部署然后其他contract链接上。

```solidity
library SafeMath {}
contract TestSafeMath {
    using SafeMath for uint; //使用方式
}
```

### Interface
声明`Interface`与其他contract交互。类似接口，Interface：

- 类似抽象函数，非实现
- 可以继承其他interfaces
- 必须为`external`
- 不能声明`constructor`
- 不能声明`state variables`

#### example
```solidity
contract Counter {
    uint public count;
    function increment() external {
        count += 1;
    }
}

interface ICounter {
    function count() external view returns (uint);
    function increment() external;
}
contract MyContract { // 实现ICounter
    function incrementCounter(address _counter) external {
        ICounter(_counter).increment(); //传入address执行该ICounter的increment方法
    }
    function getCount(address _counter) external view returns (uint) {
        return ICounter(_counter).count();
    }
}

// Uniswap example
interface UniswapV2Factory {
    function getPair(address tokenA, address tokenB) external view returns (address pair);
}

interface UniswapV2Pair {
    function getReserves() external view returns (uint112 reserve0, uint112 reserve1, uint32 blockTimestampLast);
}

contract UniswapExample {
    address private factory = 0x5C69bEe701ef814a2B6a3EDD4B1652CB9cc5aA6f;
    address private dai = 0x6B175474E89094C44Da98b954EedeAC495271d0F;
    address private weth = 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2;

    function getTokenReserves() external view returns (uint, uint) {
        address pair = UniswapV2Factory(factory).getPair(dai, weth);
        (uint reserve0, uint reserve1, ) = UniswapV2Pair(pair).getReserves();
        return (reserve0, reserve1);
    }
}


```

## Ether Coin


### Sending Ether
发送Ether的方式：
- transfer (2300 gas, throws error)
- send (2300 gas, returns bool)
- call (forward all gas or set gas, returns bool)
接收Ether的方式，至少有如下一个函数：
- receive() external payable ，msg.data为空
- fallback() external payable ，msg.data非空

推荐结合**重入守卫**，使用`call`方法。

重入守卫的方式：
1. 调用其他contract前生效状态改变
2. 使用`re-entrancy guard modifier`


```solidity
contract ReceiveEther {
    /*
    Which function is called, fallback() or receive()?

           send Ether
               |
         msg.data is empty?
              / \
            yes  no
            /     \
receive() exists?  fallback()
         /   \
        yes   no
        /      \
    receive()   fallback()
    */
    // Function to receive Ether. msg.data must be empty
    receive() external payable {}

    // Fallback function is called when msg.data is not empty
    fallback() external payable {}

    function getBalance() public view returns (uint) {
        return address(this).balance;
    }
}

contract SendEther {
    function sendViaTransfer(address payable _to) public payable {
        _to.transfer(msg.value); //not recommended
    }
    function sendViaSend(address payable _to) public payable {
        bool sent = _to.send(msg.value);  //not recommended
        require(sent, "Failed to send Ether");
    }
    function sendViaCall(address payable _to) public payable {  // recommended
        (bool sent, bytes memory data) = _to.call{value: msg.value}("");
        require(sent, "Failed to send Ether");
    }
}
```

### ether and wei
单位 1 ehter == 1e18 wei

- uint public oneWei = 1 wei; //1 wei is equal to 1
- uint public oneEther = 1 ether; // 1 ether is equal to 10^18 wei


### Gas计算
**gas = gas spent * gas price  ether**

- `gas spent` 交易中花费的gas总量
- `gas price` gas单价

区块中，`gas price`越高有更高的优先权，未用完退回。

#### Gas Limit
花费gas的上下界

- `gas limit` 交易时最大gas量，自定义设定
- `block gas limit` 块中允许的最大gas量, 私链设定

### Gas Saving Techniques
一些节省gas的技术：
- 用calldata替换内存存储
- 将state variable以memory载入
- 用`++i`替换循环`i++`
- 缓存数组元素
- 缩短链路

### keccak256
计算输入的Keccak-256哈希值：
- 把输入转为离散非重id
- Commit-Reveal scheme
- 压缩加密签名
